# PRUEBAS DEL EXTRACT

Hago aquí las cosas porque es mucho más cómodo para testear, luego lo pasaré al extract.py

La siguiente sección es la dada por el profe. Importa paquetes y se conecta a la base de datos mediante la variable `conn`

In [386]:
from pathlib import Path
import psycopg2
import pandas as pd
# https://pygrametl.org
from pygrametl.datasources import CSVSource, SQLSource
from typing import Generator

In [3]:
# Connect to the PostgreSQL source
path = Path("db_conf.txt")
if not path.is_file():
    raise FileNotFoundError(f"Database configuration file '{path.absolute()}' not found.")
try:
    parameters = {}
    # Read the database configuration from the provided txt file, line by line
    with open(path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            parameters[line.split('=', 1)[0]] = line.split('=', 1)[1].strip()
    conn = psycopg2.connect(
        dbname=parameters['dbname'],
        user=parameters['user'],
        password=parameters['password'],
        host=parameters['ip'],
        port=parameters['port']
    )
    cursor = conn.cursor()
    print("¡Conectado!")

    
except psycopg2.Error as e:
    print(e)
    raise ValueError(f"Unable to connect to the database: {parameters}")
except Exception as e:
    print(e)
    raise ValueError(f"Database configuration file '{path.absolute()}' not properly formatted (check file 'db_conf.example.txt'.")

¡Conectado!


Una vez hecho esto tenemos dos variables importantes: 
- `conn` representa la conexión a la base de datos
- `cursor` es el objeto mediante el cual hacemos las queries

Este código siguiente te dice qué tablas hay dentro del AIMS y el AMOS

In [387]:
cursor.execute( """ 
                SELECT table_schema, table_name
                FROM information_schema.tables
                WHERE table_schema IN ('AIMS', 'AMOS')
                ORDER BY table_schema, table_name
                """
)

tablas = cursor.fetchall()
    
print("Tablas que tienes:")
for tabla in tablas: print(tabla)

Tablas que tienes:
('AIMS', 'flights')
('AIMS', 'maintenance')
('AIMS', 'slots')
('AMOS', 'attachments')
('AMOS', 'forecastedorders')
('AMOS', 'maintenanceevents')
('AMOS', 'operationinterruption')
('AMOS', 'postflightreports')
('AMOS', 'technicallogbookorders')
('AMOS', 'workorders')
('AMOS', 'workpackages')


Este código siguiente sirve para extraer elementos de la tabla `AIMS.flights`. Las comillas esas raras `\"` sirven para que el programa detecte bien mayúsculas y minúsculas. 

In [ ]:

cursor.execute("""SELECT * FROM \"AIMS\".\"flights\" """) #Select de todos los elementos
batch = cursor.fetchmany(10) #Función iteradora que va devolviendo elementos cada vez que la llamas
print("Primeros 10 datos extraídos")

In [114]:
df = pd.DataFrame.from_records(batch) #Transformarlo a pandas y printearlo bonitamente
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,XY-ACK,2023-01-01 03:23:56,2023-01-01 04:23:56,Flight,230101-TIA-GVA-4486-XY-ACK,TIA,GVA,2023-01-01 03:28:56,2023-01-01 04:28:56,False,None,138,3,3
1,2,XY-ACK,2023-01-01 06:36:40,2023-01-01 08:36:40,Flight,230101-VRN-AOI-1291-XY-ACK,VRN,AOI,2023-01-01 06:38:40,2023-01-01 08:38:40,False,None,132,3,2
2,3,XY-ACK,2023-01-02 04:42:43,2023-01-02 08:42:43,Flight,230102-LGW-ANR-8878-XY-ACK,LGW,ANR,2023-01-02 04:42:43,2023-01-02 08:42:43,False,None,103,4,2
3,4,XY-ACK,2023-02-17 20:19:21,2023-02-17 23:19:21,Flight,230217-GYD-TBS-9247-XY-ACK,GYD,TBS,2023-02-17 20:23:21,2023-02-17 23:23:21,False,None,142,4,2
4,5,XY-ACK,2023-02-18 11:31:17,2023-02-18 13:31:17,Flight,230218-CHQ-PEG-4126-XY-ACK,CHQ,PEG,2023-02-18 11:46:17,2023-02-18 13:46:17,False,None,111,3,2
5,6,XY-ACK,2023-02-19 04:43:46,2023-02-19 08:43:46,Flight,230219-EMA-PDL-9646-XY-ACK,EMA,PDL,2023-02-19 06:01:46,2023-02-19 10:01:46,False,48,111,4,2
6,7,XY-ACK,2023-02-19 11:10:11,2023-02-19 14:10:11,Flight,230219-FCO-FAE-9628-XY-ACK,FCO,FAE,2023-02-19 14:12:11,2023-02-19 11:12:11,False,None,91,4,3
7,8,XY-ACK,2023-02-19 19:41:26,2023-02-19 20:41:26,Flight,230219-KGS-BHX-9784-XY-ACK,KGS,BHX,2023-02-19 19:45:26,2023-02-19 20:45:26,False,None,148,4,2
8,9,XY-ACK,2023-02-20 05:52:45,2023-02-20 08:52:45,Flight,230220-TLL-KRK-3745-XY-ACK,TLL,KRK,2023-02-20 06:07:45,2023-02-20 09:07:45,False,None,113,4,3
9,10,XY-ACK,2023-02-20 19:39:03,2023-02-20 20:39:03,Flight,230220-BIO-AMS-2423-XY-ACK,BIO,AMS,2023-02-20 19:40:03,2023-02-20 20:40:03,False,None,129,4,3


Ahora queremos hacer una función que devuelva un objeto iterador que vaya dando filas elemento por elemento. Esta función se la daremos al transform para que pueda ir extrayendo los datos a su ritmo, en forma de flujo, sin tener que cargarlo todo de golpe en la memoria local. 

In [388]:

def yield_rows(cursor: psycopg2.extensions.cursor, batch_size:int, esquema:str, tabla:str, columns:str = "*"):
    '''Extrae y devuelve los elementos de la tabla {esquema}.{tabla} de forma iterativa. '''

    cursor.execute( f"""SELECT {columns} FROM \"{esquema}\".\"{tabla}\" """) #Hacer el select de todo

    while True: #Ir extrayendo y devolviendo batches de elementos 
        
        batch = cursor.fetchmany(batch_size)
        if not batch: break  # No hay más datos
        yield batch


Esto está bien pero se puede mejorar. El principal problema es que no se cierra el cursor. El programa puede terminar por el break, por otra interrupción o porque el usuario deje de usar el iterador o lo elimine, pero en todo caso se tiene que cerrar. Esta implementación sí lo hace: 

In [389]:
def yield_rows(cursor: psycopg2.extensions.cursor, batch_size:int, esquema:str, tabla:str, columns:str = "*") -> Generator:
    '''Extrae y devuelve los elementos de la tabla {esquema}.{tabla} de forma iterativa. '''

    try:
        cursor.execute(f"""SELECT {columns} FROM "{esquema}"."{tabla}" """)

        while True:
            batch = cursor.fetchmany(batch_size)
            if not batch: break  # No hay más datos
            yield batch
    
    finally:
        cursor.close()
        print(f"Cursor cerrado para {esquema}.{tabla}")


Utilización de la función anterior:

In [383]:
iterator_flights = yield_rows(cursor, 10, "AIMS", "flights")

In [384]:
#Si le das varias veces a este bloque verás que te va devolviendo datos nuevos cada vez
pd.DataFrame.from_records(next(iterator_flights))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,XY-ACK,2023-01-01 03:23:56,2023-01-01 04:23:56,Flight,230101-TIA-GVA-4486-XY-ACK,TIA,GVA,2023-01-01 03:28:56,2023-01-01 04:28:56,False,None,138,3,3
1,2,XY-ACK,2023-01-01 06:36:40,2023-01-01 08:36:40,Flight,230101-VRN-AOI-1291-XY-ACK,VRN,AOI,2023-01-01 06:38:40,2023-01-01 08:38:40,False,None,132,3,2
2,3,XY-ACK,2023-01-02 04:42:43,2023-01-02 08:42:43,Flight,230102-LGW-ANR-8878-XY-ACK,LGW,ANR,2023-01-02 04:42:43,2023-01-02 08:42:43,False,None,103,4,2
3,4,XY-ACK,2023-02-17 20:19:21,2023-02-17 23:19:21,Flight,230217-GYD-TBS-9247-XY-ACK,GYD,TBS,2023-02-17 20:23:21,2023-02-17 23:23:21,False,None,142,4,2
4,5,XY-ACK,2023-02-18 11:31:17,2023-02-18 13:31:17,Flight,230218-CHQ-PEG-4126-XY-ACK,CHQ,PEG,2023-02-18 11:46:17,2023-02-18 13:46:17,False,None,111,3,2
5,6,XY-ACK,2023-02-19 04:43:46,2023-02-19 08:43:46,Flight,230219-EMA-PDL-9646-XY-ACK,EMA,PDL,2023-02-19 06:01:46,2023-02-19 10:01:46,False,48,111,4,2
6,7,XY-ACK,2023-02-19 11:10:11,2023-02-19 14:10:11,Flight,230219-FCO-FAE-9628-XY-ACK,FCO,FAE,2023-02-19 14:12:11,2023-02-19 11:12:11,False,None,91,4,3
7,8,XY-ACK,2023-02-19 19:41:26,2023-02-19 20:41:26,Flight,230219-KGS-BHX-9784-XY-ACK,KGS,BHX,2023-02-19 19:45:26,2023-02-19 20:45:26,False,None,148,4,2
8,9,XY-ACK,2023-02-20 05:52:45,2023-02-20 08:52:45,Flight,230220-TLL-KRK-3745-XY-ACK,TLL,KRK,2023-02-20 06:07:45,2023-02-20 09:07:45,False,None,113,4,3
9,10,XY-ACK,2023-02-20 19:39:03,2023-02-20 20:39:03,Flight,230220-BIO-AMS-2423-XY-ACK,BIO,AMS,2023-02-20 19:40:03,2023-02-20 20:40:03,False,None,129,4,3


Ahora queremos hacer la función final de extract. Usaremos la función yield_rows. Tiene que hacer esto: 

- Crear el cursor para conectarse a la base de datos original de DBeaver (AIMS y AMOS). Al final con un cursor no basta porque varios iteradores no pueden tener el mismo cursor sin interferir. Entonces cada tabla tiene que tener su propio cursor. 
- Por cada tabla relevante, generará un iterador que devuelva las cosas relevantes mediante la función `yield_rows`. Se necesita una forma de configurar qué tablas y columnas son relevantes, es por esto que la funció stc guarda todas las tuplas de (Schema, Table, Column) que se consultarán. Si un cierto elemento de `stc` fuera `('AIMS', 'slots', 'schedulearrival, kind')` entonces solo se extraerían estas dos columnas de esta tabla. 
- Finalmente devolverá un diccionario con todos los iteradores. La key del diccionario será el nombre de la tabla correspondiente
- Hará algo parecido para los archivos .csv pero eso está por ver 

In [391]:


def extract() -> dict[str, Generator]:

    iterators_dict:dict[str, Generator] = {}
    batch_size:int = 100

    #Schemas, Tables and Columns
    stc:list[tuple[str, str, str]] = [  ('AIMS', 'flights', '*'),
                                        ('AIMS', 'maintenance', '*'),
                                        ('AIMS', 'slots', '*'),
                                        ('AMOS', 'attachments', '*'),
                                        ('AMOS', 'forecastedorders', '*'),
                                        ('AMOS', 'maintenanceevents', '*'),
                                        ('AMOS', 'operationinterruption', '*'),
                                        ('AMOS', 'postflightreports', '*'),
                                        ('AMOS', 'technicallogbookorders', '*'),
                                        ('AMOS', 'workorders', '*'),
                                        ('AMOS', 'workpackages', '*')  ]
    
    for schema, table, column in stc: 
        cursor = conn.cursor()
        iterators_dict[ f"{schema}.{table}" ] = yield_rows(cursor, batch_size, schema, table, column)
    
    return iterators_dict



Uso de la función anterior. Aquí obtenemos todos los iteradores y usamos uno de ellos. 

In [392]:
diccionario = extract()

In [393]:
diccionario

{'AIMS.flights': <generator object yield_rows at 0x00000295314760E0>,
 'AIMS.maintenance': <generator object yield_rows at 0x0000029531475EA0>,
 'AIMS.slots': <generator object yield_rows at 0x0000029531474A60>,
 'AMOS.attachments': <generator object yield_rows at 0x0000029531476320>,
 'AMOS.forecastedorders': <generator object yield_rows at 0x0000029531476200>,
 'AMOS.maintenanceevents': <generator object yield_rows at 0x0000029531476680>,
 'AMOS.operationinterruption': <generator object yield_rows at 0x00000295314767A0>,
 'AMOS.postflightreports': <generator object yield_rows at 0x00000295314768C0>,
 'AMOS.technicallogbookorders': <generator object yield_rows at 0x00000295314769E0>,
 'AMOS.workorders': <generator object yield_rows at 0x0000029531476B00>,
 'AMOS.workpackages': <generator object yield_rows at 0x0000029531476C20>}

In [394]:

next(diccionario['AMOS.attachments'])


[('018c772f-f87a-410b-96f9-0762d6adbd25', 20001),
 ('581bb3bb-b9f0-4fae-8b8c-116d1bf85c51', 20001),
 ('a84edda3-fbbb-4e45-bbb1-54c5ab8c01b4', 20004),
 ('99a57b4e-956a-4104-9024-d31969c10f5c', 20006),
 ('ffd27c78-84d7-484a-acd0-e9bbbfe3dec7', 20009),
 ('2038ca98-5888-4982-8721-6410bf570468', 20009),
 ('0cd8c42a-6c70-471b-910a-68cdc124fcb5', 20010),
 ('9217ca61-be67-4f8d-ab1f-3d826dc5133f', 20010),
 ('c5d98231-39f5-48b7-85ab-6c63eeddb9b5', 20011),
 ('32884f14-6a6a-4d24-8744-b5975d42a6df', 20011),
 ('59ff0947-d3e8-4051-922f-6d047665a211', 20015),
 ('313cadb7-86a6-43b8-a035-d031138c0952', 20015),
 ('45c633da-3449-4b58-b661-fb4f8a8ffab7', 20016),
 ('dca42a75-2415-4346-ae31-394a0ff09aa2', 20016),
 ('ae5f885f-337d-4662-a7ef-0385447d2f67', 20017),
 ('a5ce4e2b-3850-4f00-b3bc-705e6d234832', 20017),
 ('54f375e6-ad7d-4e6f-af47-a037622dccd2', 20018),
 ('347bd16b-9da1-4f31-b068-a39ffd35cb01', 20019),
 ('66f46974-909e-46dc-be5c-9e99433d85ad', 20022),
 ('f4147799-3ff0-4548-b34e-276989bd5f24', 20022),


In [395]:
query = f"""
        WITH 
            atomic_data_utilization AS (
                SELECT
                    CASE 
                        WHEN f.aircraftregistration in ('{"','".join(aircrafts.get("Airbus", []))}') THEN 'Airbus'
                        WHEN f.aircraftregistration in ('{"','".join(aircrafts.get("Boeing", []))}') THEN 'Boeing'
                        ELSE f.aircraftregistration
                        END AS manufacturer, 
                    DATE_PART('year', f.scheduleddeparture)::text AS year,
                    CAST(SUM(CASE WHEN f.cancelled 
                        THEN 0
                        ELSE EXTRACT(EPOCH FROM f.actualarrival-f.actualdeparture) / 3600
                        END) AS numeric) AS flightHours,
                    CAST(SUM(CASE WHEN f.cancelled 
                        THEN 0
                        ELSE 1
                        END) AS numeric) AS flightCycles
                FROM "AIMS".flights f
                GROUP BY manufacturer, YEAR
                ),
            atomic_data_reporting AS (
                SELECT
                    CASE 
                        WHEN f.aircraftregistration in ('{"','".join(aircrafts.get("Airbus", []))}') THEN 'Airbus'
                        WHEN f.aircraftregistration in ('{"','".join(aircrafts.get("Boeing", []))}') THEN 'Boeing'
                        ELSE f.aircraftregistration
                        END AS manufacturer, 
                    DATE_PART('year', f.reportingdate)::text AS year,
                    f.reporteurclass AS role,
                    COUNT(*) AS counter
                FROM "AMOS".postflightreports f
                GROUP BY manufacturer, year, role
                )
        SELECT f1.manufacturer, f1.year, f1.role,
            1000*ROUND(f1.counter/f2.flightHours, 3) AS RRh,
            100*ROUND(f1.counter/f2.flightCycles, 2) AS RRc              
        FROM atomic_data_reporting f1
            JOIN atomic_data_utilization f2 ON f2.manufacturer = f1.manufacturer AND f1.year = f2.year
        ORDER BY f1.manufacturer, f1.year, f1.role;
        """

NameError: name 'aircrafts' is not defined